In [ ]:
from sympy import *
from IPython.display import display, Latex
init_printing() 

def display_eq(lhs,rhs,op="=",space=""):
    display(Latex(rf"${space} {latex(lhs)} {op} {latex(rhs)} $"))



def analyze_series(fx,fxs,x,verb=1,factor=1):
    '''
    Extract the coefficients of each order of a fxs=series(fx,x,0,n)
    fx=(fx_[0]+ fx_[1]*x +fx_[2]*1/2*x**2+fx_[3]*1/6*x**3 ... + O(x**n))
    Input:
        fx: sympy Function("name")(x) 
        fxs: sympy series of a function in x at x=0
        x:  sympy Symbol for the dependent variable 
        verb: Verbosity: 0: none, 1: output, 2: input and output
    output:
        dictionary with 0 coefficients excluded,
            "xord" : array of the order of each term in x 
            "xout" : array of x**ord[i]
            "x"    : from input
            "fx"   : from input
            "fxs"  : from input
            "fsymb": array of symbols for coeffient names fx.name_[xord[i]]
            "fsimp": simplified symbolic function fx=sum_i fx_[ord[i]]*x**ord[i]/factorial(ord[i])
            "fcoef": symbolic expressions for the coefficients
    '''
    
    
    oexist=fxs.getn() is not None
    fxp = expand(fxs.removeO(),x)
    pcoef=Poly(fxp,x).all_coeffs() # 0 is x^0
    maxord=len(pcoef)
    xord   =[None]*(maxord+1)
    xout   =[None]*(maxord+1)
    fxsymb =[None]*(maxord+1)
    fxcoef =[None]*(maxord+1)
    if(verb > 1): 
        print("input:")
        display_eq(fx,fxs*factor)
    for l in range(0,maxord):
        coef=simplify(pcoef[maxord-1-l]*factorial(l))*factor
        if (coef != 0):
            xord[l]=l
            fxsymb[l]=Symbol(f"{fx.name}_[{l}]")
            xout[l]=Rational(1,factorial(l))*x**l
            fxcoef[l]=coef
        else:
            fxcoef[l]=None
    if(oexist):
        xout[maxord]=1
        xord[maxord]=maxord
        fxsymb[maxord]=Order(x**maxord)
        fxcoef[maxord]=Order(x**maxord)
    for var in (xord,xout,fxsymb,fxcoef):
        var[:]= list(filter(lambda item: (item is not None),var))
    fxsimp=sum([x*y for x,y in zip(xout,fxsymb)])
    if(verb >-1):
        print("output:")
        display_eq(fx,fxsimp)
    if(verb >0 ):
        print("   with terms:")
        for l in range(0,len(xord)-(1 if oexist else 0)):
            display_eq(fxsymb[l],fxcoef[l],space="\qquad")
    
    # return the simplified function and the terms
    return {"xord":xord,  # array of the order in x
            "xout":xout,  # array of x**ord
            "x": x,    # input
            "fx": fx,
            "fxs": fxs,
            "fsymb":fxsymb,  # array of coefficient names f_[i]
            "fsimp":fxsimp,
            "fcoef":fxcoef}

def init_series(fx,x,maxord,xord_in=[],withO=True):
    if len(xord_in)==0 :
        xord=[i for i in range(0,maxord+1)]
    else:
        xord=xord_in
    xout=[x**i/factorial(i) for i in xord]
    fxsymb=[None]*(len(xord))
    fxcoef=[None]*(len(xord))
    for l in range(0,len(xord)):
            fxsymb[l]=Symbol(f"{fx.name}_[{xord[l]}]")
    if(withO):
        xout[-1]=1
        xord[-1]=maxord
        fxsymb[-1]=Order(x**maxord)
        fxcoef[-1]=Order(x**maxord)    
    return {"xord":xord,  # array of the order in x
            "xout":xout,  # array of x**ord
            "x": x,   # Symbol of dependent variable
            "fx": fx,  # Function object
            "fxs": None,
            "fsymb":fxsymb,
            "fsimp": sum([x*y for x,y in zip(xout,fxsymb)]),
            "fcoef":fxcoef,
            }

 
y=Symbol("y")
a=Symbol("alpha")

g=Function("g")(y)
gs=a*y**2
#gs=series(g,y,0,4)
#gs=gs #-gs.args[2-1]
g_simp=analyze_series(g,gs,y,verb=2)
display_eq(g,g_simp["fsimp"])

iser=init_series(g,y,4)
display_eq(g,iser["fsimp"])

iser=init_series(g,y,6,xord_in=[2,3,5,6],withO=False)
display_eq(g,iser["fsimp"])


In [ ]:
# order of the expansion in X1,X2,lambda
ord=5


In [ ]:

r = Symbol('rho', real=True,positive=True)
t = Symbol('theta', real=True)
z = Symbol('zeta', real=True)


phif=Function('Phi')(r)
phi_series=init_series(phif,r,2,xord_in=[2],withO=False)
phi=phi_series["fsimp"]
display_eq(phif,phi)
chif=Function('chi')(r)
chi_series=init_series(chif,r,2*(ord+2),xord_in=[i for i in range(2,2*(ord+2),2)])
chi=chi_series["fsimp"]
display_eq(chif,chi)


def gvec_taylor_series(fx,ord):
    fx_series=init_series(fx,r,ord+1)
    fx_series["fsymb"][0]= Function(fx.name+'_[0]',real=True)(z)
    for l in range(1,ord+1):
        fx_series["fsymb"][l]= Function(fx.name+f"_[{l}]",real=True)(t,z)
    fx_series["fsimp"]=sum([x*y for x,y in zip(fx_series["xout"],fx_series["fsymb"])])
    return fx_series

X1f = Function('X^1',real=True)(r,t,z)
X2f = Function('X^2',real=True)(r,t,z)
LAf = Function('lambda',real=True)(r,t,z)

X1_series=gvec_taylor_series(X1f,ord); X1=X1_series["fsimp"]; display_eq(X1f,X1)
X2_series=gvec_taylor_series(X2f,ord); X2=X2_series["fsimp"]; display_eq(X2f,X2)
LA_series=gvec_taylor_series(LAf,ord); LA=LA_series["fsimp"]; display_eq(LAf,LA)


In [ ]:
#Jacobian

Jh=Symbol("J_h",real=True,positive=True)
Jpf=Function("J_p",real=True,positive=True)(r,t,z)
display_eq(Jf,Jpf*Jh)
display_eq(Jpf,X1f.diff(r)*X2f.diff(t)-X1f.diff(t)*X2f.diff(r))
Jp=X1.diff(r)*X2.diff(t)-X1.diff(t)*X2.diff(r)
Jps=series(Jp,r,0,n=ord)
#display_eq(Jpf,Jps)
Jf=Function("J",real=True,positive=True)(r,t,z)
J_series=analyze_series(Jf,Jps,r,factor=Jh)

Js=J_series["fsimp"]







In [ ]:
Btf=Function('B^theta')(r,t,z)

display_eq(Btf,1/Jf*(chif.diff(r)-phif.diff(r)*LAf.diff(z)))

Bt=1/(Js)*(chi.diff(r)-phi.diff(r)*LA.diff(z))
Bt_series=analyze_series(Btf,series(Bt,r,0,n=ord-1),r)

Bts=Bt_series["fsimp"]

In [ ]:
Bzf=Function('B^zeta')(r,t,z)

display_eq(Bzf,1/Jf*phif.diff(r)*(1+LAf.diff(t)))

Bz=1/(Js)*(phi.diff(r)) #+chi.diff(r)*LA.diff(z))

Bz_series=analyze_series(Bzf,series(Bz,r,0,n=ord-1),r)

Bzs=Bz_series["fsimp"]

In [ ]:
Gh_11=Symbol("G^h_1_1",real=True)
Gh_12=Symbol("G^h_1_2",real=True)
Gh_13=Symbol("G^h_1_3",real=True)
Gh_22=Symbol("G^h_2_2",real=True)
Gh_23=Symbol("G^h_2_3",real=True)
Gh_33=Symbol("G^h_3_3",real=True)

gttf=Function("g_theta_theta")(r,t,z)
display_eq(gttf,(   X1f.diff(t)*X1f.diff(t)*Gh_11
                 +2*X1f.diff(t)*X2f.diff(t)*Gh_12
                 +  X2f.diff(t)*X2f.diff(t)*Gh_22
                ))
gtt=(   X1.diff(t)*X1.diff(t)*Gh_11
     +2*X1.diff(t)*X2.diff(t)*Gh_12
     +  X2.diff(t)*X2.diff(t)*Gh_22)
gtt_series=analyze_series(gttf,series(gtt,r,0,n=ord-1),r)
gtts=gtt_series["fsimp"]



In [ ]:
gtzf=Function("g_theta_zeta")(r,t,z)
display_eq(gtzf,(   X1f.diff(t)*X1f.diff(z)*Gh_11
                 +2*X1f.diff(t)*X2f.diff(z)*Gh_12
                 +2*X1f.diff(t)*          1*Gh_13  #dz(z)=1
                 +  X2f.diff(t)*X2f.diff(z)*Gh_22
                 +2*X2f.diff(t)*          1*Gh_23 #dz(z)=1
                ))
gtz=(   X1.diff(t)*X1.diff(z)*Gh_11
     +2*X1.diff(t)*X2.diff(z)*Gh_12
     +2*X1.diff(t)*         1*Gh_13  #dz(z)=1
     +  X2.diff(t)*X2.diff(z)*Gh_22
     +2*X2.diff(t)*         1*Gh_23 #dz(z)=1
    )
gtz_series=analyze_series(gtzf,series(gtz,r,0,n=ord-1),r)
gtzs=gtz_series["fsimp"]


In [ ]:
gzzf=Function("g_zeta_zeta")(r,t,z)
display_eq(gzzf,(   X1f.diff(z)*X1f.diff(z)*Gh_11
                 +2*X1f.diff(z)*X2f.diff(z)*Gh_12
                 +2*X1f.diff(z)*          1*Gh_13  #dz(z)=1
                 +  X2f.diff(z)*X2f.diff(z)*Gh_22
                 +2*X2f.diff(z)*          1*Gh_23 #dz(z)=1
                 +Gh_33
                ))
gzz=(   X1.diff(z)*X1.diff(z)*Gh_11
     +2*X1.diff(z)*X2.diff(z)*Gh_12
     +2*X1.diff(z)*         1*Gh_13  #dz(z)=1
     +  X2.diff(z)*X2.diff(z)*Gh_22
     +2*X2.diff(z)*         1*Gh_23 #dz(z)=1
     + Gh_33
    )
gzz_series=analyze_series(gzzf,series(gzz,r,0,n=ord-1),r)
gzzs=gzz_series["fsimp"]

In [ ]:
# norm of magnetic field:

Bf=Function("B",real=True,positive=True)(r,t,z)
display_eq(Bf,Btf*Btf*gttf+Btf*Bzf*gtzf*2+Bzf*Bzf*gzzf)
B=Bts*Bts*gtts+Bts*Bzs*gtzs*2+Bzs*Bzs*gzzs
Bt_series=analyze_series(Bf,series(B,r,0,n=ord-1),r)